## Depletion module testing

I will use this notebook to understand the functionality of OpenMC's ``deplete`` module

### 1. ``Integrator``
The ``Integrator`` class is where the `integrate()` function that a user would call to run a depletion simulation lives. The `integrate()` function does several things:

1. Gets the initial nuclide concentration (``with self.operator as conc:``)
2. Gets the source rate (power or neutron flux), time step size, and step index the depletion calculation (``for i, (dt, source_rate) in enumerate(self):``)
3. Solves the transport equation (``conc, res = self._get_bos_data_from_operator(i, source_rate, conc)``)
4. Solves the bateman equations (``proc_time, conc_list, res_list = self(conc, res.rates, dt, source_rate, i)``)
5. Saves the results in an HDF5 file.

In the following cells, I will describe how each of these steps are done in detail.